In [1]:
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import holidays
import warnings
from merge_transformer import MergeTransformer
from create_db import create_db

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def check_csv_correct(X):
    
    if X.isna().sum().all() != 0:
        return False
    return True

In [3]:
def merge_dfs(database, submission_dir):
    
    X = database['AirportStatistics']
    
    print("X airportstats cols", X.head())
    print('\n\n\n\n')
    print("date ", database['Date'].head())
    print('\n\n\n\n')

    merge_transform = MergeTransformer(X_ext=database['Date'], 
                                       how='left', 
                                       on=['DateOfDeparture'])
    X = merge_transform.fit_transform(X)
        
    print("X", X.columns)
    print('\n\n\n\n')
    print("airport", database['Airport'].columns)
    print('\n\n\n\n')
    
    merge_transform = MergeTransformer(
        X_ext=database['Airport'], 
        cols_to_rename={'iata': 'AirPort'},
        how='inner', on=['AirPort'])
    X = merge_transform.fit_transform(X)
    
    print("All good til here")
    print('\n\n\n\n')

    print("Statefeatures", database['StateFeatures'].head())
    print("\n\n\n\n\n")
    print("X ", X.columns)
    X.to_csv('../data/XMergedStateFeatures.csv')
    database['StateFeatures'].to_csv('../data/StateFeatures.csv')
    
    merge_transform = MergeTransformer(
        X_ext=database['StateFeatures'],
        cols_to_keep=['state', 'year', 'month', 'UnemploymentRate', 'holidays', 'GDP_per_cap'],
        cols_to_rename={'Abbreviation': 'state'},
        how='left',
        on=['year', 'month', 'day', 'state'])

    X = merge_transform.fit_transform(X)
    
    print("X ", X.columns)
    
    all_check = check_csv_correct(X)
    
    if all_check:
        X.to_csv(submission_dir + 'external_data.csv')
    else:
        raise Exception("Sorry will not create CSV as Database has Nans or some other shit. Try Again!")
        
    return all_check

In [4]:
def create_external_data(submission_dir):
    
    database = create_db()
    merge_dfs(database, submission_dir)

In [5]:
create_external_data('../submissions/test/')

Cols to rename =  {'name': 'municipality'}
Cols to keep =  ['name', 'pop2010']
Goes in if
new cols to keep =  ['pop2010', 'municipality']
Cols to rename =  {'2012': 'GDP_per_cap'}
Cols to keep =  ['State', '2012']
Goes in if
new cols to keep =  ['State', 'GDP_per_cap']
Cols to rename =  {'date': 'DateOfDeparture', 'is_vacation': 'school_holidays'}
Cols to keep =  ['DateOfDeparture', 'school_holidays']
new cols to keep =  ['DateOfDeparture', 'school_holidays']
X airportstats cols   AirPort  Max TemperatureC  Mean TemperatureC  Min TemperatureC  Dew PointC  \
0     ATL                35                 29                24          21   
1     ATL                36                 29                22          17   
2     ATL                35                 29                23          17   
3     ATL                27                 24                22          22   
4     ATL                26                 24                22          23   

   MeanDew PointC  Min DewpointC  M